In [1]:
!pip freeze | grep scilit-learn

In [2]:
!python -V

Python 3.9.12


In [4]:
import pickle
import pandas as pd

In [15]:
!pip install pyarrow

     |████████████████████████████████| 40.8 MB 9.9 MB/s eta 0:00:01 MB 9.9 MB/s eta 0:00:04     |███████████▉                    | 15.1 MB 9.9 MB/s eta 0:00:03      | 30.4 MB 9.9 MB/s eta 0:00:02


In [11]:
with open('model.bin', 'rb') as f_in:
    dv, model = pickle.load(f_in)

/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator DictVectorizer from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(
/home/codespace/anaconda3/lib/python3.9/site-packages/sklearn/base.py:329: UserWarning: Trying to unpickle estimator LinearRegression from version 1.5.0 when using version 1.0.2. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/modules/model_persistence.html#security-maintainability-limitations
  warnings.warn(


In [17]:
categorical = ['PULocationID', 'DOLocationID']

def read_data(filename):
    df = pd.read_parquet(filename)
    
    df['duration'] = df.tpep_dropoff_datetime - df.tpep_pickup_datetime
    df['duration'] = df.duration.dt.total_seconds() / 60

    df = df[(df.duration >= 1) & (df.duration <= 60)].copy()

    df[categorical] = df[categorical].fillna(-1).astype('int').astype('str')
    
    return df

In [18]:
df = read_data('https://d37ci6vzurychx.cloudfront.net/trip-data/yellow_tripdata_2023-03.parquet')

In [26]:
df

,VendorID,tpep_pickup_datetime,tpep_dropoff_datetime,passenger_count,trip_distance,RatecodeID,store_and_fwd_flag,PULocationID,DOLocationID,payment_type,...,mta_tax,tip_amount,tolls_amount,improvement_surcharge,total_amount,congestion_surcharge,Airport_fee,duration,year,month
0,2,2023-03-01 00:06:43,2023-03-01 00:16:43,1.0,0.00,1.0,N,238,42,2,...,0.5,0.00,0.0,1.0,11.10,0.0,0.00,10.000000,2023,3
1,2,2023-03-01 00:08:25,2023-03-01 00:39:30,2.0,12.40,1.0,N,138,231,1,...,0.5,12.54,0.0,1.0,76.49,2.5,1.25,31.083333,2023,3
2,1,2023-03-01 00:15:04,2023-03-01 00:29:26,0.0,3.30,1.0,N,140,186,1,...,0.5,4.65,0.0,1.0,28.05,2.5,0.00,14.366667,2023,3
3,1,2023-03-01 00:49:37,2023-03-01 01:01:05,1.0,2.90,1.0,N,140,43,1,...,0.5,4.10,0.0,1.0,24.70,2.5,0.00,11.466667,2023,3
4,2,2023-03-01 00:08:04,2023-03-01 00:11:06,1.0,1.23,1.0,N,79,137,1,...,0.5,2.44,0.0,1.0,14.64,2.5,0.00,3.033333,2023,3
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3403761,2,2023-03-31 23:24:25,2023-03-31 23:40:54,NaN,3.16,NaN,None,163,75,0,...,0.5,4.23,0.0,1.0,20.36,NaN,NaN,16.483333,2023,3
3403762,2,2023-03-31 23:24:50,2023-04-01 00:04:12,NaN,6.89,NaN,None,125,198,0,...,0.5,8.98,0.0,1.0,53.90,NaN,NaN,39.366667,2023,3
3403763,2,2023-03-31 23:26:31,2023-03-31 23:49:39,NaN,4.01,NaN,None,50,224,0,...,0.5,0.00,0.0,1.0,28.02,NaN,NaN,23.133333,2023,3
3403764,2,2023-03-31 23:07:51,2023-03-31 23:15:56,NaN,1.31,NaN,None,113,158,0,...,0.5,3.50,0.0,1.0,16.01,NaN,NaN,8.083333,2023,3


In [19]:
dicts = df[categorical].to_dict(orient='records')
X_val = dv.transform(dicts)
y_pred = model.predict(X_val)

In [21]:
import numpy as np

std_dev = np.std(y_pred)

std_dev

6.247488852238703

In [24]:

df['tpep_pickup_datetime'] = pd.to_datetime(df['tpep_pickup_datetime'])
df['year'] = df['tpep_pickup_datetime'].dt.year
df['month'] = df['tpep_pickup_datetime'].dt.month



In [27]:
df['ride_id'] = df['year'].map("{:04d}".format) + '/' + df['month'].map("{:02d}".format) + '_' + df.index.astype('str')

In [45]:
df_result = df[['ride_id']].copy()
df_result['predictions'] = y_pred

In [46]:
df_result.to_parquet(
    'df_output.parquet',
    engine='pyarrow',
    compression=None,
    index=False
)

In [47]:
import os

file_size = os.path.getsize('df_output.parquet')
print(f"The file size is {file_size} bytes")

The file size is 68641796 bytes


SyntaxError: invalid syntax (637305517.py, line 1)